In [31]:
import numpy as np
from collections import defaultdict
from tqdm import tqdm

In [41]:
def load_data(file_name):
    data_arr = []
    label_arr = []
    fr = open(file_name)

    for i, line in enumerate(tqdm(fr.readlines())):
        cur_line = line.strip().split(',')
        # 此外将数据进行了二值化处理，大于128的转换成1，小于的转换成0，方便后续计算
        data_arr.append([int(int(num) > 128) for num in cur_line[1:]])
        label_arr.append(int(cur_line[0]))
    return data_arr, label_arr

In [33]:
def majorClass(label):
    '''
    找到当前标签集中数目最大的标签
    '''
    class_dict = defaultdict(int)  # 建立一个字典，用于计数
    for i in range(len(label)):
        class_dict[label[i]] += 1
    # 对字典依据 值 进行降序排序
    class_sort = sorted(class_dict.items(), key=lambda x: x[1], reverse=True)
    # 返回占数目最多的标签, 返回的是label
    return class_sort[0][0]

In [34]:
def getSubDataArr(train_data, train_label, A, a):
    # 根据特征值进行划分左右子树
    ret_data = []
    ret_label = []
    for i in range(len(train_data)):
        if train_data[i][A] == a:
            # 特征值由于二值化了，只可能是0或1
            # 将该样本的第A个特征切割掉，放入返回的数据集和中
            ret_data.append(train_data[i][0:A] + train_data[i][A + 1:])
            # 列表＋列表 = 列表
            ret_label.append(train_label[i])
    return ret_data, ret_label

In [35]:
def calc_H_D(train_label):
    '''
    计算数据集D的经验熵
    label: 某个数据集的label
    '''
    H_D = 0  # 初始化为0
    # 先找出哪些label出现过，这样保证经验熵计算log时始终有意义
    label_set = set([label for label in train_label])

    for i in label_set:
        # 计算|Ck|/|D|
        p = train_label[train_label == i].size / train_label.size

        H_D += -1 * p * np.log2(p)
    return H_D

In [36]:
def calc_H_D_A(dev_feature, train_label):
    H_D_A = 0
    # 找出feature可取的所有值
    feature_set = set([label for label in dev_feature])

    # 对于每一个特征值，遍历计算条件经验熵
    for i in feature_set:
        # 计算 H(D|A)
        rate = dev_feature[dev_feature == i].size / dev_feature.size
        H_D_A += rate * calc_H_D(train_label[dev_feature == i])
    return H_D_A

In [37]:
def calcBestFeature(train_data, train_label):
    '''
    计算信息增益最大的特征
    train_data ： 当前数据集
    '''
    train_data = np.array(train_data)
    train_label = np.array(train_label)
    feature_num = train_data.shape[1]  # 获取特征的数目

    maxG_D_A = -1  # 初始化最大信息增益
    max_feature = -1  # 初始化最大信息增益的特征

    # 1. 计算数据集D的经验熵 H(D)
    H_D = calc_H_D(train_label)

    # 2. 计算经验条件熵
    for feature in range(feature_num):

        # 提取所有样本在当前feature的列表, 加快运行速度
        train_data_devide_by_feature = np.array(train_data[:, feature].flat)

        # 计算信息增益 G(D|A) = H(D) - H(D|A)
        G_D_A = H_D - calc_H_D_A(train_data_devide_by_feature, train_label)

        if G_D_A > maxG_D_A:
            maxG_D_A = G_D_A
            max_feature = feature

    return max_feature, maxG_D_A


In [38]:
def createTree(*dataset):
    # :param dataSet:(trainDataList， trainLabelList) <<-- 元祖形式
    # :return:新的子节点或该叶子节点的值
    # * 的作用：在函数定义中，收集所有位置参数到一个新的元组，并将整个元组赋值给变量args
    epsilon = 0.1

    # 若信息增益 < 阈值Epsilon，则返回 T
    train_data = dataset[0][0]
    train_label = dataset[0][1]
    # 将标签放入一个set里面，如果当前都是同一个标签，则返回该标签
    class_dict = {i for i in train_label}
    if len(class_dict) == 1:
        return train_label[0]

    # 如果A为空集，则返回, 即没有特征可以供分解了
    if len(train_data[0]) == 0:
        # #即如果已经没有特征可以用来再分化了，返回占大多数的类别
        return majorClass(train_label)

    # 计算信息增益，找到信息增益最大的feature Ag
    Ag, EpsilonGet = calcBestFeature(train_data, train_label)

    # 如果信息增益值小于阈值epsilon
    if EpsilonGet < epsilon:
        return majorClass(train_label)

    # 对Ag的每一个值，将Ag=ai将D分割为若干非空子集合Di,将Di实例最大的类
    # 作为标记，构建子节点
    tree_dict = {Ag: {}}
    tree_dict[Ag][0] = createTree(getSubDataArr(train_data, train_label, Ag, 0))
    tree_dict[Ag][1] = createTree(getSubDataArr(train_data, train_label, Ag, 1))

    return tree_dict


In [39]:
def predict(test_data, tree):
    '''
    test data: 待预测的样本
    tree: 构建好的决策树
    '''
    while True:
        # 使用下行这种方式读取key和value
        (key, value), = tree.items()  # TODO 为什么要这样读取？ 记住就行了， 可以以 {73: {0: {74:6}}} 为例试一试
        if type(tree[key]).__name__ == 'dict':
            data_val = test_data[key]  # 这是该样本的某个特征值
            # 不断地删除已经用掉的特征，保证索引相对位置的一致性
            del test_data[key]  # 删除该样本的feature
            tree = value[data_val]
            if type(tree).__name__ == 'int':
                return tree
        else:
            # 返回分类值
            return value


In [40]:
def model_test(test_data, test_label, tree):
    error_cnt = 0
    for i in tqdm(range(len(test_data))):
        if test_label[i] != predict(test_data[i], tree):
            error_cnt += 1
    return 1 - error_cnt / len(test_data)


In [42]:
train_data, train_label = load_data('../../data/Mnist/train.csv')
test_data, test_label = load_data('../../data/Mnist/test.csv')

100%|██████████| 10000/10000 [00:02<00:00, 4996.30it/s]


In [30]:
print('create tree ing ...')
tree = createTree((train_data, train_label))
acc = model_test(test_data, test_label, tree)
print(acc)

(60000,)
